In [1]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
import os, sys, glob, warnings, glob    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *
import joblib
import importlib
from importlib import reload
sys.path.append("/project/rrg-mdiamond/owhgabri/pyTracker")
sys.path.insert(1, "/project/rrg-mdiamond/owhgabri/pyTracker")
os.chdir('/project/rrg-mdiamond/owhgabri/pyTracker')
print(os.getcwd())
print(joblib.__version__)

/project/6049244/owhgabri/pyTracker
1.3.2


In [2]:
import scipy
import copy as cp

# ROOT
import ROOT as root

# Matplotlib
import matplotlib.pyplot as plt
from matplotlib import collections, colors, transforms

import pprint

Welcome to JupyROOT 6.28/10


In [3]:
from tracker import kalmanfilter as KF
from tracker import utilities as Util
from tracker import trackfinder as TF
from tracker import datatypes
from tracker.datatypes import *


reload(TF)
reload(Util)

<module 'utilities' from '/project/rrg-mdiamond/owhgabri/pyTracker/tracker/utilities.py'>

In [1]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
import os, sys, glob, warnings, glob    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *
import joblib
import importlib
from importlib import reload
sys.path.append("/home/owhgabri/My_GitHub/MATHUSLA_JupyterNoteBooks/tracker")
sys.path.insert(1, "/home/owhgabri/My_GitHub/MATHUSLA_JupyterNoteBooks/tracker")
os.chdir('/home/owhgabri/My_GitHub/pyTracker')
print(os.getcwd())
print(joblib.__version__)

/home/owhgabri/My_GitHub/pyTracker
1.4.2


In [2]:
import scipy
import copy as cp

# ROOT
import ROOT as root

# Matplotlib
import matplotlib.pyplot as plt
from matplotlib import collections, colors, transforms

%matplotlib inline
%config InlineBackend.figure_format='retina'
# %matplotlib widget

import pprint

Welcome to JupyROOT 6.26/14


# Useful Functions #

In [4]:
def TrackNumberVeto(vertex, n):
    """
    Returns True if the vertex has at least n tracks
    """
    if len(vertex.tracks) >= n:
        return True
    return False
        

# Background #

In [5]:
data_top_dir = f"/project/rrg-mdiamond/data/MATHUSLA/simulation/run-2024-07-mathusla40-full/DigiOutput"
pathList=[]

for rootFile, dirs, files in os.walk(data_top_dir):
    for filename in files:
        if "stat0io_MuSim-noise10-layers4-eff1.00" in filename:
            pathList.append(os.path.join(rootFile, filename))

print("BACKGROUND")
print(len(pathList))


30


## Survival Rate (Event Level) ## 

In [8]:
print("Survival Rate: Event Level")
nEvents = 0
nTracks = [3, 4]
nVetoedStrict = []
nVetoedLenient = []
for n in nTracks:
    nVetoedStrict.append(0)
    nVetoedLenient.append(0)
for f in pathList:
    events=joblib.load(f)
    file_vertices = events["vertices"]
    for i in range(len(file_vertices)):
        vertices = file_vertices[i]
        if len(vertices) == 0:
            continue
        nEvents += 1
        survivedStrict = []
        survivedLenient = []
        for n in nTracks:
            survivedStrict.append(True)
            survivedLenient.append(False)
        for vertex in vertices:
            for j in range(len(nTracks)):
                if survivedStrict and not TrackNumberVeto(vertex, nTracks[j]):
                    survivedStrict[j] = False
                if TrackNumberVeto(vertex, nTracks[j]):
                    survivedLenient[j] = True
        for j in range(len(survivedStrict)):
            if survivedStrict[j]:
                nVetoedStrict[j] += 1
            if survivedLenient[j]:
                nVetoedLenient[j] += 1
            
print("Number of events:", nEvents)
for i in range(len(nTracks)):
    print("Fraction Surviving if requiring " + str(nTracks[i]) + " tracks (strict)", nVetoedStrict[i]/nEvents)
    print("Fraction Surviving if requiring " + str(nTracks[i]) + " tracks (lenient)", nVetoedLenient[i]/nEvents)



Number of events: 1182
Fraction Surviving if requiring 3 tracks (strict) 0.02707275803722504
Fraction Surviving if requiring 3 tracks (lenient) 0.04145516074450085
Fraction Surviving if requiring 4 tracks (strict) 0.0
Fraction Surviving if requiring 4 tracks (lenient) 0.0008460236886632825


# Signal #

In [5]:
data_top_dir = f"/project/rrg-mdiamond/data/MATHUSLA/simulation/LLPrun-2024-07-08/Reconstruction"
pathList=[]

for rootFile, dirs, files in os.walk(data_top_dir):
    for filename in files:
        if ".pkl" in filename:
            pathList.append(os.path.join(rootFile, filename))
print("SIGNAL")
print(len(pathList))

SIGNAL
5


## Survival Rate (Event Level) ## 

In [ ]:
print("Survival Rate: Event Level")
nEvents = 0
nTracks = [3, 4]
nVetoedStrict = []
nVetoedLenient = []
for n in nTracks:
    nVetoedStrict.append(0)
    nVetoedLenient.append(0)
for f in pathList:
    events=joblib.load(f)
    file_vertices = events["vertices"]
    for i in range(len(file_vertices)):
        vertices = file_vertices[i]
        if len(vertices) == 0:
            continue
        nEvents += 1
        survivedStrict = []
        survivedLenient = []
        for n in nTracks:
            survivedStrict.append(True)
            survivedLenient.append(False)
        for vertex in vertices:
            for j in range(len(nTracks)):
                if survivedStrict and not TrackNumberVeto(vertex, nTracks[j]):
                    survivedStrict[j] = False
                if TrackNumberVeto(vertex, nTracks[j]):
                    survivedLenient[j] = True
        for j in range(len(survivedStrict)):
            if survivedStrict[j]:
                nVetoedStrict[j] += 1
            if survivedLenient[j]:
                nVetoedLenient[j] += 1
            
print("Number of events:", nEvents)
for i in range(len(nTracks)):
    print("Fraction Surviving if requiring " + str(nTracks[i]) + " tracks (strict)", nVetoedStrict[i]/nEvents)
    print("Fraction Surviving if requiring " + str(nTracks[i]) + " tracks (lenient)", nVetoedLenient[i]/nEvents)

